In [2]:
import requests
import pandas
from urllib.parse import quote

# API CALL with multiple games

In [33]:


import requests
import json
from urllib.parse import quote

# Steam API URLs
REVIEW_URL = "https://store.steampowered.com/appreviews/"
STORE_URL = "https://store.steampowered.com/api/appdetails?appids="

# List of Game IDs (Example: Add more game IDs here)
games = [
    1677280, 570, 730, 578080, 440, 271590, 252490, 620, 394690, 1328670,
    275850, 1817070, 413150, 271590, 1281930, 578080, 227300, 381210, 892970, 
    526870, 1085660, 359550, 1454400, 292030, 582010, 374320, 236850, 381210,
    289070, 1091500, 1466860, 1174180, 377160, 227300, 244850, 548430, 239140,
    1245620, 583950, 678960, 359320, 823500, 2050650, 578080, 250900, 1237970,
    493340, 813780, 620980, 638970, 282800, 310560, 236430, 1084600, 239160,
    1589560, 234650, 359870, 291550, 282800, 408410, 105600, 668580, 640820,
    362490, 636480, 1517290, 739630, 1018020, 1454400, 1030840, 739630, 813780,
    554920, 897450, 1238860, 292030, 678960, 739630, 814380, 892970, 236850, 
    431240, 1238860, 218620, 239140, 550650, 602960, 1187140, 1446780, 216150,
]

games = [str(game_id) for game_id in games]

# API Parameters
APPENDIX = "?json=1&language=english"
CURSOR_APPENDIX = "&cursor="

# Store all reviews
newlist = []

# Function to get game details (Name & Genre)
def get_game_details(game_id):
    response = requests.get(STORE_URL + game_id)
    if response.status_code == 200:
        game_data = response.json()
        if game_id in game_data and "data" in game_data[game_id]:
            game_info = game_data[game_id]["data"]
            game_name = game_info.get("name", "Unknown Game")
            genres = [genre["description"] for genre in game_info.get("genres", [])]
            genre = ", ".join(genres) if genres else "Unknown Genre"
            return game_name, genre
    return "Unknown Game", "Unknown Genre"

# Loop through multiple games
for game_id in games:
    print(f"Scraping reviews for game ID: {game_id}")

    # Get game details
    game_name, genre = get_game_details(game_id)

    # Reset cursor and result list for each game
    cursor = "*"
    result = []

    # Fetch up to 5 pages of reviews per game
    for i in range(20):
        url2 = REVIEW_URL + game_id + APPENDIX + "&language=english" + CURSOR_APPENDIX + cursor
        print(url2)
        response = requests.get(url2)

        if response.status_code == 200:
            reviews_data = response.json()
        else:
            print(f"Error {response.status_code} for game ID {game_id}")
            break

        try:
            cursor = quote(reviews_data["cursor"])
        except KeyError:
            result.append(reviews_data)
            break
        result.append(reviews_data)

    # Store reviews with game details
    for i in result:
        if "reviews" in i:
            for j in i["reviews"]:
                temp_list = [game_name, j["review"], j["voted_up"], genre]
                newlist.append(temp_list)

# Print sample output
print("\nExample Review Data:")
for item in newlist[:5]:  # Show first 5 reviews
    print(item)


Scraping reviews for game ID: 1677280
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=*
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFZqTIAAAAB2rc7QBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFZ%2BTCAAAAB4pIfRBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFaAAAAAAAB2g4DOBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFaWgAAAAAB67e/LBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFbAAAAAAAB9r63QBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFdhheAAAAB4zJTLBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=

In [35]:
df = pandas.DataFrame(newlist, columns = ["Game", "Review","Recommended", "Genre"])
df["Genre"] = "Strategy"
df

,Game,Review,Recommended,Genre
0,Company of Heroes 3,"Right, I believe I have played this game enoug...",True,Strategy
1,Company of Heroes 3,I Pre-ordered the premium edition version of t...,False,Strategy
2,Company of Heroes 3,They listed my GPU as supported and then quiet...,False,Strategy
3,Company of Heroes 3,CoH3 is where it's at. I've been playing CoH s...,True,Strategy
4,Company of Heroes 3,"An underwhelming, expensive cash grab. Very lo...",False,Strategy
...,...,...,...,...
32531,MapleStory,NOT WORTH PLAYING ANYMORE.\n\nGMS is pretty mu...,False,Strategy
32532,MapleStory,This game is great for someone who just wants ...,False,Strategy
32533,MapleStory,Looks bad at first but when you get in and pla...,True,Strategy
32534,MapleStory,What's this? I was just munching some pasture ...,True,Strategy


In [36]:
df.head()

,Game,Review,Recommended,Genre
0,Company of Heroes 3,"Right, I believe I have played this game enoug...",True,Strategy
1,Company of Heroes 3,I Pre-ordered the premium edition version of t...,False,Strategy
2,Company of Heroes 3,They listed my GPU as supported and then quiet...,False,Strategy
3,Company of Heroes 3,CoH3 is where it's at. I've been playing CoH s...,True,Strategy
4,Company of Heroes 3,"An underwhelming, expensive cash grab. Very lo...",False,Strategy


In [32]:
df.to_csv("Steam_Reviews.csv", mode = "a", index = False)